In [10]:
import os
import glob
import numpy as np
import time
from helper import R_to_angle
from params import par
from torchvision import transforms
from PIL import Image
import torch
import math
from preprocess import *
%load_ext autoreload
%autoreload 2

In [11]:
train_video = ['00', '02', '08', '09', '06', '04', '10']
# train_video = ['00', '02', '08' ]
image_path_list = []
for folder in train_video:
    image_path_list += glob.glob('KITTI/images/{}/*.png'.format(folder))

In [12]:
# image_path_list

In [14]:
img_as_img = Image.open(image_path_list[10])
mean_np = [0, 0, 0]
mean_tensor = [0, 0, 0]

In [15]:
to_tensor = transforms.ToTensor()

In [16]:
img_as_tensor = to_tensor(img_as_img)
img_as_tensor -= 0.5

In [17]:
img_as_np = np.array(img_as_img)
img_as_np = np.rollaxis(img_as_np, 2, 0)

In [18]:
img_as_np.shape

(3, 376, 1241)

In [20]:
par.img_h


184

In [22]:
par.img_w

608

In [19]:
cnt_pixels = 0
cnt_pixels += img_as_np.shape[1]*img_as_np.shape[2]

In [54]:
for c in range(3):
        mean_tensor[c] += float(torch.sum(img_as_tensor[c]))
        mean_np[c] += float(np.sum(img_as_np[c]))
mean_tensor =  [v / cnt_pixels for v in mean_tensor]
mean_np = [v / cnt_pixels for v in mean_np]
print('mean_tensor = ', mean_tensor)
print('mean_np = ', mean_np)

mean_tensor =  [-0.09540397142671919, -0.07572827214990485, -0.0719063309686123]
mean_np =  [103.17198510123956, 108.18928840845578, 109.16388636480532]


In [55]:
new_size = (par.img_w, par.img_h)
transform_ops = []
transform_ops.append(transforms.Resize((new_size[0], new_size[1])))
transformer = transforms.Compose(transform_ops)
normalizer = transforms.Normalize(mean=list(par.img_means), std=list(par.img_stds))

In [56]:
par.img_means

(0.19007764876619865, 0.15170388157131237, 0.10659445665650864)

In [57]:
img_as_tensor_reduced=transformer(img_as_tensor)
# img_as_tensor_reduced = normalizer(img_as_tensor_reduced)

/home/randychen233/anaconda3/envs/ICON_lab/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [58]:
mean_np = [0, 0, 0]
mean_tensor = [0, 0, 0]

In [59]:
cnt_pixels = 0
cnt_pixels += img_as_np.shape[1]*img_as_np.shape[2]
for c in range(3):
        mean_tensor[c] += float(torch.sum(img_as_tensor_reduced[c]))
mean_tensor =  [v / cnt_pixels for v in mean_tensor]
mean_np = [v / cnt_pixels for v in mean_np]
print('mean_tensor = ', mean_tensor)

mean_tensor =  [-0.022905312615860794, -0.018168077444836866, -0.01720136185656407]


In [60]:
use_cuda = True
device = torch.device("cuda" if use_cuda and torch.cuda.is_available() else "cpu")


In [14]:
from data_helper import *
videos_to_test = ['07']

In [15]:
seq_len = int((par.seq_len[0]+par.seq_len[1])/2)
overlap = seq_len - 1
print('seq_len = {},  overlap = {}'.format(seq_len, overlap))
batch_size = par.batch_size

seq_len = 6,  overlap = 5


In [16]:
for test_video in videos_to_test:
    df = get_data_info(folder_list=[test_video], seq_len_range=[seq_len, seq_len], overlap=overlap, sample_times=1, shuffle=False, sort=False)

Folder 07 finish in 0.021198511123657227 sec


In [17]:
dataset = ImageSequenceDataset(df, par.resize_mode, (par.img_w, par.img_h), par.img_means, par.img_stds, par.minus_point_5)

In [18]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
n_batch = len(dataloader)

In [35]:
for i, batch in enumerate(dataloader):
        print('{} / {}'.format(i, n_batch), end='\r', flush=True)
        _, x, y = batch
        # print(x.shape, y.shape)
        print(x.min(),x.std())
        
        break
        

tensor(-2.5831) tensor(1.1679)


In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [49]:
epsilons = [0, .05, .1, .15, .2, .25, .3]
pretrained_model = "data/lenet_mnist_model.pth"
use_cuda=True
# Set random seed for reproducibility
torch.manual_seed(42)

In [65]:
# LeNet Model definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

# MNIST Test dataset and dataloader declaration
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,)),
            ])),
        batch_size=1, shuffle=True)


In [66]:
use_cuda = True
device = torch.device("cuda" if use_cuda and torch.cuda.is_available() else "cpu")

In [74]:
def test( model, device, test_loader, epsilon ):

    # Accuracy counter
    correct = 0
    adv_examples = []

    # Loop over all examples in test set
    for data, target in test_loader:

        # Send the data and label to the device
        data, target = data.to(device), target.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

        # If the initial prediction is wrong, don't bother attacking, just move on
        if init_pred.item() != target.item():
            continue

        # Calculate the loss
        loss = F.nll_loss(output, target)

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Collect ``datagrad``
        data_grad = data.grad.data

        # Restore the data to its original scale
        data_denorm = denorm(data)

        # Call FGSM Attack
        perturbed_data = fgsm_attack(data_denorm, epsilon, data_grad)

        # Reapply normalization
        perturbed_data_normalized = transforms.Normalize((0.1307,), (0.3081,))(perturbed_data)

        # Re-classify the perturbed image
        output = model(perturbed_data_normalized)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        if final_pred.item() == target.item():
            correct += 1
            # Special case for saving 0 epsilon examples
            if epsilon == 0 and len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        else:
            # Save some adv examples for visualization later
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )

    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader))
    print(f"Epsilon: {epsilon}\tTest Accuracy = {correct} / {len(test_loader)} = {final_acc}")

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

In [70]:
# restores the tensors to their original scale
def denorm(batch, mean=[0.1307], std=[0.3081]):
    """
    Convert a batch of tensors to their original scale.

    Args:
        batch (torch.Tensor): Batch of normalized tensors.
        mean (torch.Tensor or list): Mean used for normalization.
        std (torch.Tensor or list): Standard deviation used for normalization.

    Returns:
        torch.Tensor: batch of tensors without normalization applied to them.
    """
    if isinstance(mean, list):
        mean = torch.tensor(mean).to(device)
    if isinstance(std, list):
        std = torch.tensor(std).to(device)

    # for channel in range(1): #for KITTI datasets
        # batch[:, channel, :, :] = batch[:, channel, :, :] * std[channel] + mean[channel] 
    return batch * std.view(1, -1, 1, 1) + mean.view(1, -1, 1, 1)



In [75]:
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [76]:
accuracies = []
examples = []

# Run test for each epsilon
for eps in epsilons:
    acc, ex = test(model, device, test_loader, eps)
    accuracies.append(acc)
    examples.append(ex)

NameError: name 'model' is not defined